# Extreme Learning Machine (ELM)

## Importing the libraries

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

## Importing the dataset

In [70]:
df = pd.read_csv("./Dataset/Concrete_Strength.csv")

def strength_class(x):
    if x < 20:
        return 0  # Low
    elif x < 40:
        return 1  # Medium
    else:
        return 2  # High

df["Strength_Class"] = df["Concrete_compressive_strength"].apply(strength_class)

# Features & target
X = df.drop(columns=["Concrete_compressive_strength", "Strength_Class"]).fillna(0.0)
y = df["Strength_Class"]

## Splitting the dataset into the Training set and Test set

In [71]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

## Feature Scaling

In [72]:
# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

## Training the ELM model on the Training set

In [73]:
class ELMClassifier:
    def __init__(self, input_dim, hidden_dim=500, activation='relu'):
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.activation = activation
        self.W = np.random.randn(input_dim, hidden_dim)
        self.b = np.random.randn(hidden_dim)

    def _activation(self, X):
        if self.activation == 'sigmoid':
            return 1 / (1 + np.exp(-X))
        elif self.activation == 'relu':
            return np.maximum(0, X)
        else:
            return X

    def fit(self, X, y):
        y_onehot = np.zeros((y.size, y.max() + 1))
        y_onehot[np.arange(y.size), y] = 1
        H = self._activation(np.dot(X, self.W) + self.b)
        self.beta = np.linalg.pinv(H) @ y_onehot

    def predict(self, X):
        H = self._activation(np.dot(X, self.W) + self.b)
        y_pred_prob = H @ self.beta
        return np.argmax(y_pred_prob, axis=1)


elm_clf = ELMClassifier(input_dim=X_train_scaled.shape[1], hidden_dim=500, activation='relu')
elm_clf.fit(X_train_scaled, y_train)

## Predicting the Test set results

In [74]:
y_pred = elm_clf.predict(X_test_scaled)

print("\n📊 ELM Classification Report:")
print(classification_report(y_test, y_pred, target_names=["Low", "Medium", "High"]))



📊 ELM Classification Report:
              precision    recall  f1-score   support

         Low       0.62      0.82      0.70        39
      Medium       0.75      0.78      0.76        91
        High       0.81      0.63      0.71        76

    accuracy                           0.73       206
   macro avg       0.73      0.74      0.73       206
weighted avg       0.75      0.73      0.73       206



## Making the Confusion Matrix

In [75]:
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print(f"Accuracy: {accuracy_score(y_test, y_pred) * 100:.2f}%")

Confusion Matrix:
 [[32  7  0]
 [ 9 71 11]
 [11 17 48]]
Accuracy: 73.30%


# ADMM Top 5 features(SVM)

## ADMM Feature Selection

In [76]:
def select_top_k_via_admm(X, y, lam=0.01, rho=1.0, iters=200, k=5):
    Xn = (X - X.mean(axis=0)) / (X.std(axis=0) + 1e-12)
    Xn = Xn.values if hasattr(Xn, "values") else np.asarray(Xn)
    yv = y.values if hasattr(y, "values") else np.asarray(y)

    n, p = Xn.shape
    w = np.zeros(p)
    z = np.zeros(p)
    u = np.zeros(p)

    XtX = Xn.T @ Xn
    Xty = Xn.T @ yv
    I = np.eye(p)

    def soft_threshold(v, t):
        return np.sign(v) * np.maximum(np.abs(v) - t, 0.0)

    for _ in range(iters):
        w = np.linalg.solve(XtX + rho * I, Xty + rho * (z - u))
        z = soft_threshold(w + u, lam / rho)
        u = u + (w - z)

    idx = np.argsort(-np.abs(z))[:k]
    selected_cols = list(X.columns[idx])
    return selected_cols

top5_cols = select_top_k_via_admm(X_train, y_train, lam=0.01, rho=1.0, iters=300, k=5)
print("Selected (Top-5 ADMM):", top5_cols)

X = X[top5_cols]

Selected (Top-5 ADMM): ['Cement (component 1)(kg in a m^3 mixture)', 'Age (day)', 'Blast Furnace Slag (component 2)(kg in a m^3 mixture)', 'Fly Ash (component 3)(kg in a m^3 mixture)', 'Water  (component 4)(kg in a m^3 mixture)']


## Splitting the dataset into the Training set and Test set

In [77]:
# Full feature matrix (no target columns)
X_all = X
y_cls_all = df["Strength_Class"].to_numpy()
y_reg_all = df["Concrete_compressive_strength"].to_numpy()  # for feature selection (regression)


X_train, X_test, y_train_cls, y_test_cls, y_train_reg, y_test_reg = train_test_split(X_all, y_cls_all, y_reg_all, test_size=0.2, random_state=42, stratify=y_cls_all)


## Feature Scaling

In [78]:
clf_scaler = StandardScaler()
X_train_clf = clf_scaler.fit_transform(X_train)
X_test_clf  = clf_scaler.transform(X_test)

## Training the ELM model on the Training set

In [79]:
elm = ELMClassifier(input_dim=X_train_clf.shape[1], hidden_dim=500, activation='relu')
elm.fit(X_train_clf, y_train_cls)

y_pred = elm.predict(X_test_clf)

## Predicting the Test set results

In [80]:
print("\n📊 ELM Classification Report:")
print(classification_report(y_test_cls, y_pred, target_names=["Low", "Medium", "High"]))



📊 ELM Classification Report:
              precision    recall  f1-score   support

         Low       0.54      0.82      0.65        39
      Medium       0.73      0.67      0.70        91
        High       0.83      0.70      0.76        76

    accuracy                           0.71       206
   macro avg       0.70      0.73      0.70       206
weighted avg       0.73      0.71      0.71       206



## Making the Confusion Matrix

In [81]:
print("Confusion Matrix:\n", confusion_matrix(y_test_cls, y_pred))
print(f"Accuracy: {accuracy_score(y_test_cls, y_pred) * 100:.2f}%")

Confusion Matrix:
 [[32  7  0]
 [19 61 11]
 [ 8 15 53]]
Accuracy: 70.87%
